<a href="https://colab.research.google.com/github/Pranjalya/fake-news-twitter/blob/get-label/Fact_Check_News_Labelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Label Extractor

Downloading data.

In [2]:
!curl 'https://storage.googleapis.com/kaggle-data-sets/689642%2F1213945%2Fcompressed%2FAllTweets_Dec2019_June2020.csv.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1592040961&Signature=PnELBhPSkmNTP0vu9lQEBhLJ1vseVwKF3AbMfu8A%2BLQXHU8YIrktWe9XF5GGx%2F57RGGfGDsaMAzmXAZzbcTBCDgY9d7DsIjqZWMIKdlnOWBOpCXK1qBKMC%2FLxHadGq8Y%2BWQvKkim%2BhHuvvHiA0ig9%2B8b%2FmxmcppHWQxujeN0OZmGIuLtA5foCR2e9FfiT6tsey74%2FwBinTe8jXhDzaoz1SJq7gNMvhN595YY6pukLuhA2gdPt6tHJYDFX9gWBQaYDNQzi4yOdMBbUhUPBjW231b6zVKy180ToMJxFhel%2FdHDGPDbZKPXtfPnx9rEb%2BCIe3GnhfCSwbZe%2Bf0c0VmaQg%3D%3D' -H 'authority: storage.googleapis.com' -H 'upgrade-insecure-requests: 1' -H 'dnt: 1' -H 'user-agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36' -H 'sec-fetch-dest: document' -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9' -H 'sec-fetch-site: cross-site' -H 'sec-fetch-mode: navigate' -H 'sec-fetch-user: ?1' -H 'referer: https://www.kaggle.com/' -H 'accept-language: en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7' --compressed --output fact_checker.zip
!mkdir data
!unzip fact_checker.zip -d data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.1M  100 10.1M    0     0  47.0M      0 --:--:-- --:--:-- --:--:-- 47.0M
mkdir: cannot create directory ‘data’: File exists
Archive:  fact_checker.zip
  inflating: data/AllTweets_Dec2019_June2020.csv  


## Study and Analysis

In [7]:
import pandas as pd
import re
import os
import numpy as np

fcn = pd.read_csv('/content/data/AllTweets_Dec2019_June2020.csv')
# Extracting given urls in the tweet
fcn['relay_url'] = fcn['tweet'].str.extract(
    pat="(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)")
fcn.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,conversation_id,created_at,date,timezone,place,tweet,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,Region,relay_url
0,1267487446809313282,1267487446809313282,1591027530000,2020-06-01 16:05:30,UTC,NaN,"Aarti Lalchandani, principal of GVSM Medical C...",['#altnewsfactcheck'],[],717020664587227136,717020664587227136,AltNews,Alt News,1,16,https://twitter.com/AltNews/status/12674874468...,False,811,40,373,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '717020664587227136', 'username':...",NaN,NaN,NaN,NaN,Others,https://www.altnews.in/verification-speech-tar...
1,1267108282164019202,1267108282164019202,1590937130000,2020-05-31 14:58:50,UTC,NaN,Times Now played three videos during a show on...,['#altnewsfactcheck'],[],717020664587227136,717020664587227136,AltNews,Alt News,7,14,https://twitter.com/AltNews/status/12671082821...,False,278,9,92,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '717020664587227136', 'username':...",NaN,NaN,NaN,NaN,Others,https://www.altnews.in/times-now-airs-old-vide...
2,1266732527227744256,1266732527227744256,1590847543000,2020-05-30 14:05:43,UTC,NaN,A video of an incident from Surat has been sha...,['#altnewsfactcheck'],[],717020664587227136,717020664587227136,AltNews,Alt News,6,14,https://twitter.com/AltNews/status/12667325272...,False,148,1,56,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '717020664587227136', 'username':...",NaN,NaN,NaN,NaN,Others,https://www.altnews.in/video-from-surat-shared...
3,1266671072671391744,1266671072671391744,1590832891000,2020-05-30 10:01:31,UTC,NaN,A video showing poor condition of Mumbai's KEM...,['#altnewsfactcheck'],[],717020664587227136,717020664587227136,AltNews,Alt News,6,10,https://twitter.com/AltNews/status/12666710726...,False,157,5,64,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '717020664587227136', 'username':...",NaN,NaN,NaN,NaN,Others,https://www.altnews.in/video-from-mumbais-kem-...
4,1266653343767597057,1266653343767597057,1590828665000,2020-05-30 08:51:05,UTC,NaN,"Criticising journalists on COVID-19 reporting,...",['#altnewsfactcheck'],[],717020664587227136,717020664587227136,AltNews,Alt News,6,8,https://twitter.com/AltNews/status/12666533437...,False,439,25,196,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '717020664587227136', 'username':...",NaN,NaN,NaN,NaN,Others,https://www.altnews.in/solicitor-general-tusha...


In [0]:
group = fcn.groupby('username')
sample = pd.DataFrame(columns = fcn.columns)
for key, value in group:
    sample = pd.concat([sample, value.head(2)])

In [9]:
for x, y in zip(list(sample['username']), list(sample['tweet'])):
    print("{} : {}".format(x, y))

ABCFactCheck : CORRECTION: in an earlier tweet, we paraphrased @zalisteggall as saying there are 60,000 people employed in coal mining. She actually said "in fossil-fuel and coal mining". Here's our fact check on the issue (not including oil and gas numbers):  https://ab.co/2JzVUeM  #qanda
ABCFactCheck : Thanks Zali, you said (according to TVeeder): "there's 242,000 jobs in mining, of which it's only about 60,000 that are in fossil-fuel and coal mining." Apologies for the misquote.

FTR, your number (38,000 in coal mining) accords with one of the data sets in our fact check
AfpFactuel : Conversions massives à l’islam, musulmans immunisés face au virus : dans le déchaînement international de désinformation suscité par le #Covid_19, les théoriciens du complot au Moyen-Orient ont, eux, plongé dans un excès de religiosité  http://u.afp.com/3doz  #AFP
AfpFactuel : Foisonnantes sur les réseaux sociaux du monde entier, les théories complotistes présentant Bill Gates en grand orchestrateur du 

## Extracting label from AltNews

In [11]:
altnews = fcn[fcn['username']=='AltNews']
altnews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 821 entries, 0 to 820
Data columns (total 35 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               821 non-null    int64  
 1   conversation_id  821 non-null    int64  
 2   created_at       821 non-null    int64  
 3   date             821 non-null    object 
 4   timezone         821 non-null    object 
 5   place            0 non-null      object 
 6   tweet            821 non-null    object 
 7   hashtags         821 non-null    object 
 8   cashtags         821 non-null    object 
 9   user_id          821 non-null    int64  
 10  user_id_str      821 non-null    int64  
 11  username         821 non-null    object 
 12  name             821 non-null    object 
 13  day              821 non-null    int64  
 14  hour             821 non-null    int64  
 15  link             821 non-null    object 
 16  retweet          821 non-null    bool   
 17  nlikes          

In [12]:
list(altnews['tweet'])[:10]

['Aarti Lalchandani, principal of GVSM Medical College in Kanpur, was caught on camera targeting Tablighi Jamaat and the Muslim community. She first claimed that the video is morphed then said her remarks were made in anger. #AltNewsFactCheck | @ArchitMeta https://www.altnews.in/verification-speech-targeting-muslim-community-by-kanpur-medical-college-principal/?utm_source=website&utm_medium=social-media&utm_campaign=newpost\xa0…',
 'Times Now played three videos during a show on India-China tensions. The anchor said that the channel “brings these visuals” for viewers but disclaimers during the show ironically referred to them as "file" footage. #AltNewsFactCheck | @Pooja_Chaudhuri https://www.altnews.in/times-now-airs-old-videos-during-broadcast-on-recent-india-china-border-tensions/?utm_source=website&utm_medium=social-media&utm_campaign=newpost\xa0…',
 "A video of an incident from Surat has been shared with false communal narrative that a Muslim man was made to lick a Hindu's spittle

In [19]:
def getLabel(tweet: str) -> str:
    if(re.search('false', tweet)):
        return 'false'
    elif(re.search('true', tweet)):
        return 'true'
    else:
        return ''
altnews['label'] = altnews.apply(lambda row: getLabel(row['tweet']), axis=1)
altnews['label'].value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


false    408
         407
true       6
Name: label, dtype: int64

In [20]:
for x, y in zip(list(altnews['label']), list(altnews['tweet'])):
    if(x=='false' or x=='true'):
        print("{} : {}".format(x, y))

false : A video of an incident from Surat has been shared with false communal narrative that a Muslim man was made to lick a Hindu's spittle. The police has clarified that both perpetrator and victim hailed from the same community. #AltNewsFactCheck | @ArchitMeta https://www.altnews.in/video-from-surat-shared-with-false-claim-that-muslim-man-was-forced-to-lick-hindus-spit/?utm_source=website&utm_medium=social-media&utm_campaign=newpost …
false : A video showing poor condition of Mumbai's KEM hospital after staff members went on a strike is viral with the false claim that dead bodies kept lying on beds while patients slept on the floor of a Delhi hospital. #AltNewsFactCheck | @thisisjignesh https://www.altnews.in/video-from-mumbais-kem-hospital-shared-as-dead-bodies-piled-up-in-delhi-hospital/?utm_source=website&utm_medium=social-media&utm_campaign=newpost …
false : Criticising journalists on COVID-19 reporting, SG Tushar Mehta quoted a false WhatsApp forward in Supreme Court. It spoke 